In [1]:
import pandas as pd
import numpy as np
import pickle

dataframes = pd.read_pickle('recom_data/user_category_dummy3.pkl')

df_counts = dataframes['view_counts']
df_users = dataframes['users']

with open('recom_data/category_dict.pkl', 'rb') as handle:
    category_dict = pickle.load(handle)

item_similarity = pd.read_pickle('recom_data/item_similarity.pkl')

In [2]:
# IBCF에서 사용할 유저 dummy데이터입니다.
db_user_data = [[1, 'F', 26]]
df_db_user_data = pd.DataFrame(db_user_data, columns=['user', 'gender', 'age'])
new_df_users = df_users.append(df_db_user_data)

# 이 유저는 1, 2, 3, 4, 31, 32, 33번 카테고리의 제품을 봤네요..
db_view_data = [[1, 1, 4], [1, 2, 2], [1, 3, 2], [1, 4, 1], [1, 31, 1], [1, 32, 1], [1, 33, 3]]
df_db_view_data = pd.DataFrame(db_view_data, columns=['user', 'category', 'count'])
new_df_counts = df_counts.append(df_db_view_data)

view_matrix = new_df_counts.pivot(index='user', columns='category', values='count')

In [3]:
from sklearn.model_selection import train_test_split
x = new_df_counts.copy()
y = new_df_counts['user']
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.25, stratify=y)

In [4]:
def recom_category(user_id, n_items=20):
    for category in view_matrix:
        # 모든 카테고리에 대해서 예측값을 만들어줍니다.
        user_category.loc[category] = IBCF_model(user_id, category)
    category_sort = user_category.sort_values(ascending=False)[:n_items]
    # 예측값을 sorting하고.... n_items만큼만 뽑아줍니다.
    return category_sort

def IBCF_model(user, category):
    """
        예측값을 만들어주는 Model입니당
        특정 유저 id와 특정 카테고리 no를 토대로 예측값을 만들어줍니다.
        item_similarity: 미리 만들어둔 코사인 유사도로 계산한 category의 유사도입니다.
        user_category: 해당 user의 카테고리별 조회수입니다.
    """
    sim_scores = item_similarity[category]
    # 해당 카테고리와 다른 카테고리의 유사도들입니다.
    user_viewing = user_category.T
    # 해당 user의 카테고리별 조회수입니다. (계산을 위해 전치행렬로 만듭니다.)
    non_viewing_idx = user_viewing[user_viewing.isnull()].index
    # user가 조회하지 않은 카테고리의 index를 찾습니다.
    user_viewing = user_viewing.dropna()
    sim_scores = sim_scores.drop(non_viewing_idx)
    # user_viewing과 sim_scores에서 비어있는거 제거합니다. (계산속도 향상용)
    pred_viewing = np.dot(sim_scores, user_viewing) / sim_scores.sum()
    # 예측값은 유저의 조회 기록을 해당 카테고리와의 유사도로 가중평균한 값입니다.
    return pred_viewing

test_user = {
    'user_id': 1,
    'gender': 'F',
    'age': 26
}

user_category = view_matrix.loc[test_user['user_id']].copy()
# test_user의 카테고리별 조회수입니다.
pred_user_category = recom_category(user_id=test_user['user_id'], n_items=10)
# 추천 카테고리를 가져옵니다. (위 함수 참조)
recommend_category = pd.DataFrame(pred_user_category).reset_index()
recommend_category['category'] = recommend_category['category'].map(lambda x: category_dict[x])
# 카테고리 no를 카테고리명으로 바꿉니다.
print(recommend_category)

  category         1
0    여성 상의  2.349575
1   여성 운동화  2.134135
2   여성 실내화  1.906181
3    공용 시계  1.876158
4    여성 하의  1.874882
5   여성용 가방  1.874085
6     스마트폰  1.872292
7  휴대폰 배터리  1.871499
8      피쳐폰  1.869691
9      임부복  1.869640


In [5]:
# 이건 피어슨 상관계수로 한번 해봤습니다..........
item_similarity2 = pd.read_pickle('recom_data/item_similarity_pearson.pkl')

def recom_category(user_id, n_items=20):
    for category in view_matrix:
        user_category.loc[category] = IBCF_model(user_id, category)
    category_sort = user_category.sort_values(ascending=False)[:n_items]
    return category_sort

def IBCF_model(user, category):
    sim_scores = item_similarity2[category]
    user_viewing = user_category.T
    non_viewing_idx = user_viewing[user_viewing.isnull()].index
    user_viewing = user_viewing.dropna()
    sim_scores = sim_scores.drop(non_viewing_idx)
    pred_viewing = np.dot(sim_scores, user_viewing) / sim_scores.sum()
    return pred_viewing

user_category = view_matrix.loc[test_user['user_id']].copy()

pred_user_category = recom_category(user_id=test_user['user_id'], n_items=10)
recommend_category = pd.DataFrame(pred_user_category).reset_index()
recommend_category['category'] = recommend_category['category'].map(lambda x: category_dict[x])
print(recommend_category)

   category          1
0    유아동 도서  66.335469
1     필기 용품  24.386104
2     문구 용품  11.945681
3    소설/만화책   9.195280
4  기타 수납 가구   8.676177
5      전공서적   8.374187
6      BB크림   7.926797
7    바디 클렌저   7.479300
8     아동 책장   6.354860
9     기타 침구   6.016919


In [6]:
#####################################
# 이하는 Model 정확도 측정입니다.   #
#####################################

In [7]:
# RMSE 계산해주는 함수
def RMSE(y_true, y_pred):
    return np.sqrt(np.mean((np.array(y_true)- np.array(y_pred)) ** 2))

# 모델별 RMSE 계산
def score(model):
    id_pairs = zip(x_test['user'], x_test['category'])
    y_pred = np.array([model(user, category) for (user, category) in id_pairs])
    y_true = np.array(x_test['count'])
    return RMSE(y_true, y_pred)

train_view_matrix = x_train.pivot(index='user', columns='category', values='count')
view_matrix_t = np.transpose(train_view_matrix)

In [8]:
def IBCF_model(user, category):
    if category in item_similarity:
        sim_scores = item_similarity[category]
        user_viewing = view_matrix_t[user]
        non_viewing_idx = user_viewing[user_viewing.isnull()].index
        user_viewing = user_viewing.dropna()
        sim_scores = sim_scores.drop(non_viewing_idx)
        pred_viewing = np.dot(sim_scores, user_viewing) / sim_scores.sum()
    else:
        pred_viewing = 0
    return pred_viewing

# 아마..없는 값을 평균으로 안 쓰고 0으로 넣어서 오차가 큰가봐요
print(score(IBCF_model))

1.1351443883548373


In [9]:
##################################
# 언제 전환할 것인가? 선택하기   #
##################################
"""
    기존 그룹화 추천의 경우 데이터가 없는 경우는 Error가 낮게 나올 수밖에 없음
    모델과 에러가 큰 차이가 안 날 경우 전환하는 걸로 얘기해야 될 듯....
    그러니까요.. 음....
    데이터를 지금 몇개 넣어놨는데... IBCF로 예측값을 만들고...
    그 데이터랑 비교해서 Error가 어느정도 이하면 타협하고 넘어가는 걸로 합시다!
    기본 추천 시스템 vs IBCF 비교는 어렵네요.
    
    아 아닌가;;
    나중에 할게요!! 지금 중요한거아닙니당
"""

'\n    기존 그룹화 추천의 경우 데이터가 없는 경우는 Error가 낮게 나올 수밖에 없음\n    모델과 에러가 큰 차이가 안 날 경우 전환하는 걸로 얘기해야 될 듯....\n    그러니까요.. 음....\n    데이터를 지금 몇개 넣어놨는데... IBCF로 예측값을 만들고...\n    그 데이터랑 비교해서 Error가 어느정도 이하면 타협하고 넘어가는 걸로 합시다!\n    기본 추천 시스템 vs IBCF 비교는 어렵네요.\n    \n    아 아닌가;;\n    나중에 할게요!! 지금 중요한거아닙니당\n'

In [10]:
def IBCF_model(user, category):
    sim_scores = item_similarity[category]
    user_viewing = user_category.T
    non_viewing_idx = user_viewing[user_viewing.isnull()].index
    user_viewing = user_viewing.dropna()
    sim_scores = sim_scores.drop(non_viewing_idx)
    pred_viewing = np.dot(sim_scores, user_viewing) / sim_scores.sum()
    return pred_viewing

test_user = {
    'user_id': 1,
    'gender': 'F',
    'age': 26
}

user_category = view_matrix.loc[test_user['user_id']].copy()

pred_data_df = new_df_counts[new_df_counts['user'] == test_user['user_id']]
pred_data_df['pred_ibcf'] = pred_data_df['category'].apply(lambda category: IBCF_model(test_user['user_id'], category))

print(pred_data_df)

   user  category  count  pred_ibcf
0     1         1      4   2.349575
1     1         2      2   2.116761
2     1         3      2   1.933432
3     1         4      1   1.749313
4     1        31      1   1.890808
5     1        32      1   1.994277
6     1        33      3   2.214792


C:\Users\lee33\AppData\Local\Programs\Python\Python35\lib\site-packages\ipykernel_launcher.py:19: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


In [11]:
merged_df = pd.merge(new_df_counts, new_df_users, left_on='user', right_on='user')
print(merged_df)
test2222222 = (merged_df.groupby(['gender', 'age', 'category'])['count']
               .agg(['sum', 'size']))
test2222222['mean'] = test2222222['sum'] / test2222222['size']
print(test2222222)
"""
    현재 작성 중입니다... 기본 모델로 예측값 만들거에용...
"""
# pred_data_df['pred_basic'] = pred_data_df['category'].apply(lambda category: test2222222.loc['F'].loc[20].loc[category]['mean'])

print(pred_data_df)

        user  category  count gender  age
0      90001        11      0      M   10
1      90001        12      0      M   10
2      90001        42      0      M   10
3      90001        43      0      M   10
4      90001        72      1      M   10
...      ...       ...    ...    ...  ...
63132      1         3      2      F   26
63133      1         4      1      F   26
63134      1        31      1      F   26
63135      1        32      1      F   26
63136      1        33      3      F   26

[63137 rows x 5 columns]
                     sum  size   mean
gender age category                  
C      100 1          10    10  1.000
           2          10    10  1.000
           3          10    10  1.000
           4          10    10  1.000
           11         10    10  1.000
...                  ...   ...    ...
M      70  1077       61    40  1.525
           1091       33    20  1.650
           1092       39    20  1.950
           1093       39    20  1.950
           109